# Assignment: Foundations of Bayes’ theorem

Fill out the blanks as per the instructions below.

This assignment uses type hints, so make sure to stick to those.

Whenever you need to fill in a blank, we used Python's ellipsis (`...`).

# Part 1 (A): Bayes' theorem with discrete random variables

Here, we assume a discrete prior $P(\theta)$, as well as a discrete probability distribution over a few i.i.d. observations.

The goal is to manually implement functionals for computing marginal and conditional likelihoods/probability densities.

You need to show that the posterior probability $P(\theta|Y)$ is a proper probability mass function.
Choose a different number of parameters and observations.

In [17]:
# Let's define our parameters theta and their probabilities (our prior belief):
# A handful of thetas is enough.
theta: list[int] = [1, 2, 3, 4, 5]
theta_probs: list[float] = [0.1, 0.2, 0.4, 0.2, 0.1]

# Here are our observations Y (don't change!):
Y_obs = [0.5, 1.2]

# Instead of assuming some (parameterized) distribution,
# we hardcode the conditional likelihoods of Y given some theta.
# Note that P(Y|\theta) is a likelihood, so it does not represent
# (necessarily) a valid probability density (i.e., values for each
# \theta do not necessarily have to sum to 1).
P_Y_given_theta: dict[int, dict[float, float]] = {
    1: {0.5: 0.8, 1.2: 0.3},
    2: {0.5: 0.6, 1.2: 0.5},
    3: {0.5: 0.4, 1.2: 0.8},
    4: {0.5: 0.2, 1.2: 0.9},
    5: {0.5: 0.1, 1.2: 0.7}
}

## Define PMFs

For convenience, we define the PMFs for $\theta$ and $Y$ explicitly:

In [18]:
# Don't change!
def P_theta(val: float) -> float:
    assert val in theta
    idx = theta.index(val)
    return theta_probs[idx]

## Define Functions

for the likelihood $P(Y|\theta)$, the prior $P(\theta)$, and the evidence $P(Y)$.

Recall that the evidence:

$$
\begin{align}
    P(Y)&=\sum_i\,P(Y|\theta_i)\times P(\theta_i)\nonumber.
\end{align}
$$

In [19]:
# Likelihood, P(Y|\theta), now explicitly from our discrete definition:
def likelihood(Y: list[float], t: float) -> float:

    prob = 1.0
    for y in Y:
        if y in P_Y_given_theta[t]:
            prob *= P_Y_given_theta[t][y]
        else:
            return 0.0  # If y is not defined for this theta, likelihood is zero.
    return prob

# The Evidence (in this assignment, it *is* computable):
def P_Y(Y: list[float]) -> float:
    evidence = 0.0
    for t in theta:
        evidence += P_theta(t) * likelihood(Y, t)
    return evidence

# The posterior:
def P_theta_given_Y(t: float, Y: list[float]) -> float:
    return (likelihood(Y, t) * P_theta(t)) / P_Y(Y)

# Part 1(B): Bayes' theorem with continuous random variables

-------------------------

Now, we change our model a bit.
Instead of assuming a small discrete set of possible values for $\theta$, we will assume that this parameter follows a standard normal distribution.

For our actual model, we will assume another normal distribution, where the standard deviation (scale) is fixed at $\frac{3}{2}$ and the mean is set to $\theta$: $N\sim(\mu=\theta,\sigma=\frac{3}{2})$.

We will re-use the previous observations.


The evidence, defined continuously:

$$
\begin{align}
    P(Y)=\int_{\theta}\,P(Y|t)\times P(t)\,d\theta\nonumber.
\end{align}
$$

In [28]:
# Don't change!
posterior_probs = [round(P_theta_given_Y(t=t, Y=Y_obs), ndigits=5) for t in theta]
posterior_probs

[0.09412, 0.23529, 0.50196, 0.14118, 0.02745]

In [29]:
# Don't change! The result here needs to be ~1.0!
print(sum(posterior_probs))

1.0


In [22]:
from scipy.stats.distributions import norm

# Our prior:
def P_theta_continuous(val: float) -> float:
    # Use norm.pdf() to compute this.
    return norm.pdf(val, loc=0, scale=1)


In [23]:
import numpy as np
from scipy.integrate import quad
from typing import final


# Realistically, our bounds could be -10,10 (or similar), but
# scipy's quad allows to use infinity, so we'll use that, as
# it's also closer to how we would formulate this mathematically.
a, b = -np.inf, np.inf


# Our model prototype that takes a single scale parameter that
# will be held fixed for any subsequent likelihood computations.
@final
class Model:
    """Keep using this model class as-is, no need to change it."""
    def __init__(self, scale: float):
        self.scale = scale
    
    def likelihood(self, x: float, mean: float) -> float:
        return norm.pdf(x=x, loc=mean, scale=self.scale).item()
    

def likelihood_continuous(Y: list[float], t: float, model: Model) -> float:
    prob = 1.0
    for y in Y:
        prob *= norm.pdf(x=y, loc=t, scale=model.scale)
    return prob

def P_Y_continuous(Y: list[float], model: Model) -> float:
    """Use quad() to integrate."""
    from scipy.integrate import quad
    integrand = lambda t: likelihood_continuous(Y=Y, t=t, model=model) * P_theta_continuous(t)
    evidence, _ = quad(func=integrand, a=a, b=b)
    return evidence


def P_theta_given_Y_continuous(t: float, Y: list[float], evidence: float, model: Model) -> float:
    numerator = likelihood_continuous(Y=Y, t=t, model=model) * P_theta_continuous(t)
    return numerator / evidence


# The goal of this function is to assert that our posterior is
# a valid probability density that sums/integrates to 1.
def P_theta_given_Y_continuous_integral(Y: list[float], model: Model) -> float:
    evidence = P_Y_continuous(Y=Y, model=model)
    func = lambda t: P_theta_given_Y_continuous(Y=Y, t=t, model=model, evidence=evidence)
    return quad(func=func, a=a, b=b)[0]

Now we show the amount of evidence, as well as that our posterior integrates to $\approx1$:
Also, we show the amount of (log-)evidence:

In [24]:
# Don't change. Prints the log-evidence, as well as its integral (should be ~1.0).
from math import log
use_model = Model(scale=1.5)

log(P_Y_continuous(Y=Y_obs, model=use_model)),\
P_theta_given_Y_continuous_integral(Y=Y_obs, model=use_model)

(-3.1912461104301157, 0.999999999999999)

### Find and use a better model

Recall that our observations were fixed at $[0.5, 1.2]$ and we assumed our model would be a normal distribution with standard deviation $\sigma=\frac{3}{2}$.

In [25]:
Y_obs_arr = np.array(Y_obs)
Y_obs_arr.std().item(), Y_obs_arr.mean().item()

(0.35, 0.85)

However, we know that the standard deviation should likely be smaller to accommodate our data better.
What we want to show here, is that a better model (here: same as previous but with a fixed standard deviation closer to $0.35$) produces a larger **evidence**.

In [26]:
from scipy.optimize import minimize_scalar

# Use 'minimize_scalar' to find some optimal solution
optimal_scale = minimize_scalar(
    lambda s: -P_Y_continuous(Y=Y_obs, model=Model(scale=s)),
    bounds=(0.01, 1),
    method='bounded'
).x


In [27]:
# Don't change! Prints the log-evidence, as well as its integral (should be ~1.0).
better_model = Model(scale=optimal_scale)

log(P_Y_continuous(Y=Y_obs, model=better_model)),\
P_theta_given_Y_continuous_integral(Y=Y_obs, model=better_model)

(-2.360448901823369, 1.0)

# Short evaluation (write 1-2 sentences per):

1. How has the (log-)evidence changed using the optimal scale?
2. In Bayesian terms, what does this result mean?

**Answers**:

1. The log-evidence has increased significantly from approximately -3.19 (with the original scale=1.5) to approximately -2.36 (with the optimal scale). This represents an improvement of about 0.83, meaning the evidence itself more than doubled. The optimization process found an optimal scale of approximately 0.49, which is much closer to the empirical standard deviation of the observations (~0.35).
   
2. In Bayesian terms, the evidence (marginal likelihood) quantifies how well a model predicts the observed data, integrating over all possible parameter values weighted by the prior. A higher evidence indicates that the model with the optimal scale is substantially better at explaining the observations Y=[0.5, 1.2]. This demonstrates that model comparison via evidence maximization can identify better-fitting models, with the optimal scale being closer to the true data-generating process.